In [32]:
import time
import calendar
import codecs
import datetime
import json
import sys
import gzip
import string
import glob
import re
import os
import numpy as np
from os import listdir
from os.path import isfile, join
import csv
import networkx as nx
from demjson import decode
import pandas as pd
import random
from collections import defaultdict
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [33]:
globalTweetCounter = 0

timeFormat = "%Y-%m-%dT%H:%M:%S.%fZ"

In [34]:
countries=['SAfrica-community-relevant-restricted.json']

for m in range(len(countries)):
    file_string=countries[m]
    mypath='/Users/tomfw/Downloads/DataShared/'+file_string
    G=nx.Graph()
    tweetList = []
    timeList = []
    userList = []
    
    with open(mypath) as f1:
        for line in f1:
           if random.random() < 1.1: # < 1 if you want less data
                tweetObj = json.loads(line)
                currentTime = datetime.datetime.strptime(tweetObj['postedTime'], timeFormat)
                id2=int(re.findall('^.*:([0-9]+)$',str(tweetObj['actor']['id']))[0])
                G.add_node(id2, parent=True, child=False)
                for ui in tweetObj['twitter_entities']['user_mentions']:
                    id1=ui['id']
                    G.add_node(id1, child=True)
                    G.add_edge(id1,id2, posted=currentTime, message=tweetObj['body'])
                    #print id1, id2
                try:
                    if (not tweetObj['body'].lower().startswith("rt")):
                        # Increment tweet count
                        globalTweetCounter += 1
                        tweetList.append(tweetObj['body'].lower())
                        timeList.append(currentTime)
                        userList.append(tweetObj['actor']['id'])
                        #print globalTweetCounter, tweetObj['body'].lower()
                except:
                    pass


In [35]:
print np.min(timeList)
print np.max(timeList)
split = datetime.datetime(2014, 5, 25) #train/test split

2014-03-15 04:26:31
2014-06-13 22:05:22


In [36]:
print G.number_of_edges(), G.number_of_nodes(), len(timeList)

173629 26669 90502


In [37]:
for node in G.nodes():
    nTrain = 0
    nTest = 0
    for nbr in G.neighbors(node):
        if G.edge[node][nbr]['posted'] < split:
            nTrain += 1
        else:
            nTest += 1
    if nTrain < 3 or nTest < 3:
        G.remove_node(node)

In [38]:
print G.number_of_edges(), G.number_of_nodes()

21498 1267


In [47]:
jacDict = {}
adamDict = {}
nbrDict = {}
attDict = {}

test_links = {}
train_links = {}

progress = 0
for node in G.nodes():
    jacDict[node] = defaultdict(int)
    adamDict[node] = defaultdict(int)
    nbrDict[node] = defaultdict(int)
    attDict[node] = defaultdict(int)
    test_links[node] = defaultdict(int)
    train_links[node] = defaultdict(int)
    
for n1 in G.nodes():
    progress += 1
    if progress % 100 == 0:
        print progress
    for n2 in G.nodes():
        safe = True
        j = nx.jaccard_coefficient(G, [(n1, n2)])
        ai = nx.adamic_adar_index(G, [(n1, n2)])
        pa = nx.preferential_attachment(G, [(n1, n2)])
        for nbr in nx.common_neighbors(G, n1,n2):
            nbrDict[n1][n2] += 1
            nDeg = nx.degree(G, nbr)
            if nDeg < 2:
                safe = False
        for u,v,p in j:
            jacDict[n1][n2] = p
        if safe:
            for u,v,a in ai:
                adamDict[n1][n2] = a
        for u,v,p in pa:
            attDict[n1][n2] = p
            
        

100
200
300
400
500
600
700
800
900
1000
1100
1200


In [48]:
G_train = G.copy()
for u,v in G_train.edges():
    if G_train[u][v]['posted'] > split:
        G_train.remove_edge(u,v)
        
trl = 0
tel = 0
for n1 in G.nodes():
    for n2 in G.nodes():
        if G_train.has_edge(n1, n2):
            trl += 1
            train_links[n1][n2] = 1
        if G.has_edge(n1, n2):
            tel += 1
            test_links[n1][n2] = 1
            
print("%d training link, %d test links" % (trl, tel))

32828 training link, 42591 test links


In [50]:
u = []
v = []
has_links = []
jac_co = []
adam = []
att = []
nbrs = []
count = 0
for n1 in G_train.nodes():
    count += 1
    if count % 100 == 0:
        print count
    for n2 in G_train.nodes():
        if n1 != n2:
            u.append(n1)
            v.append(n2)
            has_links.append(train_links[n1][n2])
            jac_co.append(jacDict[n1][n2])
            adam.append(adamDict[n1][n2])
            att.append(attDict[n1][n2])
            nbrs.append(nbrDict[n1][n2])
df_train = pd.DataFrame()
df_train['u'] = u
df_train['v'] = v
df_train['link'] = has_links
df_train['jac'] = jac_co
df_train['adam'] = adam
df_train['nbrs'] = nbrs
df_train['att'] = att

100
200
300
400
500
600
700
800
900
1000
1100
1200


In [51]:
u = []
v = []
has_links = []
jac_co = []
adam = []
att = []
nbrs = []
count = 0
for n1 in G.nodes():
    count += 1
    if count % 100 == 0:
        print count
    for n2 in G.nodes():
        if n1 != n2 and train_links[n1][n2] == 0:
            u.append(n1)
            v.append(n2)
            has_links.append(test_links[n1][n2])
            jac_co.append(jacDict[n1][n2])
            adam.append(adamDict[n1][n2])
            att.append(attDict[n1][n2])
            nbrs.append(nbrDict[n1][n2])
df_test = pd.DataFrame()
df_test['u'] = u
df_test['v'] = v
df_test['link'] = has_links
df_test['jac'] = jac_co
df_test['adam'] = adam
df_test['att'] = att
df_test['nbrs'] = nbrs

100
200
300
400
500
600
700
800
900
1000
1100
1200


In [56]:
rf = RandomForestClassifier()
fields = ['jac', 'att', 'nbrs', 'adam']
x_train = df_train.loc[:, fields]
#x_train = np.reshape(x_train, (-1, 1))
y_train = np.reshape(df_train.loc[:,'link'], (-1, 1))
#x_test = np.reshape(df_test.loc[:, fields], (-1, 1))
x_test = df_test.loc[:, fields]
rf.fit(x_train, y_train)
pred = rf.predict_proba(x_test)

/Users/tomfw/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [67]:
print pred[0:50]

[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]


In [66]:
roc_auc_score(np.reshape(df_test.loc[:,'link'], (-1, 1)), pred)

ValueError: Found input variables with inconsistent numbers of samples: [1571514, 2]